In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import layers
from keras import optimizers
import os
import ma

Using TensorFlow backend.


In [2]:
FILE_PATH = "/Users/jasonadam/Downloads/cv_medical/processed_arrays/"
TRAINING_FILES = os.listdir(FILE_PATH)
FILES = [os.path.join(FILE_PATH, f) for f in TRAINING_FILES]

In [3]:
array_data = [np.load(f) for f in FILES]
array_data = np.array(array_data)
array_data.shape

(52, 8, 128, 128)

In [4]:
training_array = np.zeros((52, 8, 128, 128, 1))

for i in range(52):
    training_array[i, :, :, :, 0] = array_data[i, :, :, :]
    
training_array.shape

(52, 8, 128, 128, 1)

In [14]:
train_subset = training_array[0:50]
train_subset.shape

(50, 8, 128, 128, 1)

In [7]:
labels_df = pd.read_csv("/Users/jasonadam/Downloads/cv_medical/ich_reads.csv")
labels_df.head()

,patient_id,ich_flag
0,CQ500-CT-427,1
1,CQ500-CT-181,1
2,CQ500-CT-99,0
3,CQ500-CT-47,0
4,CQ500-CT-195,0


In [15]:
file_ids = [n.split(".")[0] for n in TRAINING_FILES]
file_ids = pd.DataFrame(file_ids, columns=["file_id"])
file_labels = file_ids.merge(labels_df, how="left",left_on="file_id", right_on="patient_id")
file_labels.reset_index(inplace=True)
file_labels.drop(columns=["index", "file_id"], inplace=True)

training_labels = file_labels["ich_flag"]
training_labels_subset = training_labels[0:50]
training_labels_subset.shape

(50,)

In [31]:
file_labels[0:50].groupby(["ich_flag"]).count()

,patient_id
ich_flag,
0,11
1,39


In [18]:
def model1():
    model = Sequential()
    model.add(
        layers.Conv3D(
            filters=32,
            kernel_size=5, 
            input_shape=(8, 128, 128, 1), 
            activation="relu"
        )
    )
    model.add(layers.MaxPooling3D())
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation="relu"))
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dense(1, activation="sigmoid"))
    
    model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    return model

In [19]:
model1 = model1()
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_2 (Conv3D)            (None, 4, 124, 124, 32)   4032      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 2, 62, 62, 32)     0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 246016)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               125960704 
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 126,096,321
Trainable params: 126,096,321
Non-trainable params: 0
______________________________________________________________

In [20]:
hist1 = model1.fit(train_subset, training_labels_subset, epochs=5, batch_size=10, verbose=2, shuffle=True)

Epoch 1/5
 - 20s - loss: 3.3316 - acc: 0.6200
Epoch 2/5
 - 16s - loss: 3.5073 - acc: 0.7800
Epoch 3/5
 - 16s - loss: 3.5073 - acc: 0.7800
Epoch 4/5
 - 16s - loss: 3.5073 - acc: 0.7800
Epoch 5/5
 - 16s - loss: 3.5073 - acc: 0.7800


In [24]:
hist1.history

{'loss': [3.331610143184662,
  3.50732479095459,
  3.5073248386383056,
  3.5073248863220217,
  3.5073248386383056],
 'acc': [0.6200000002980233,
  0.7799999952316284,
  0.7799999952316284,
  0.7799999833106994,
  0.7799999952316284]}